In [63]:
import tensorflow as tf
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
import numpy as np
AE_modelPath = "models/ae_credit_16_notbest_noscaler.h5"


In [64]:
tf.__version__

'2.0.0'

In [65]:
def load_credit(dataPath):
    credit = pd.read_csv(dataPath)
    credit = credit.drop(['Time'], axis=1)
    #credit['Amount'] = StandardScaler().fit_transform(credit['Amount'].values.reshape(-1,1))
    #credit['Amount'] = MinMaxScaler().fit_transform(credit['Amount'].values.reshape(-1,1))
    df_normal = credit[credit['Class'] == 0]
    df_anomaly = credit[credit['Class'] == 1]
    del df_normal['Class']
    del df_anomaly['Class']

    # DataFrame to Numpy Array
    normal = df_normal.values
    anomaly = df_anomaly.values

    return [normal, anomaly]

In [66]:
[x_normal, x_anomaly] = load_credit("creditcard.csv")

In [79]:
encoder = tf.keras.models.load_model("models/ae_credit_16_notbest_noscaler.h5")

In [80]:
#X_normal = encoder.predict(x_normal)
#X_anomaly = encoder.predict(x_anomaly)
X_normal = x_normal
X_anomaly = x_anomaly
x_size = X_normal.shape[1]
h_size = 4
y_size = 1
D = x_size
K = h_size
#g = lambda x : relu(x)                # RELU
g = lambda x: 1 / (1 + tf.exp(-x))    # Sigmoid
#g = lambda x : x 

In [81]:
theta = np.random.normal(0, 1, K + K*D + 1)
rvalue = np.random.normal(0, 1, (len(X_normal), y_size))
nu = 0.04

In [82]:
def init_weights(shape):
    weights = tf.random_normal(shape, mean=0, stddev=0.00001)
    return tf.Variable(weights)

''' Forward Propagation '''
def forwardprop(X, w_1, w_2):
    X = tf.cast(X, tf.float32)
    w_1 = tf.cast(w_1, tf.float32)
    w_2 = tf.cast(w_2, tf.float32)
    h = tf.nn.sigmoid(tf.matmul(X, w_1))
    yhat = tf.matmul(h, w_2)
    return yhat

def nnScore(X, w, V, g):
    X = tf.cast(X, tf.float32)
    w = tf.cast(w, tf.float32)
    V = tf.cast(V, tf.float32)
    return tf.matmul(g((tf.matmul(X, w))), V)

def relu(x):
    y = x
    return y

def ocnn_obj(theta, X, nu, w1, w2, g, r):
    X = tf.cast(X, tf.float32)
    w = tf.cast(w1, tf.float32)
    V = tf.cast(w2, tf.float32)

    term1 = 0.5 * tf.reduce_sum(w**2)
    term2 = 0.5 * tf.reduce_sum(V**2)
    term3 = 1/nu * tf.reduce_mean(tf.nn.relu(r - nnScore(X, w, V, g)))
    term4 = -r
    return term1 + term2 + term3 + term4


In [88]:
initial = tf.keras.initializers.RandomNormal(mean=0, stddev=0.00001)
#initial = tf.keras.initializers.glorot_normal()

In [89]:
W1 = tf.Variable(initial_value=initial((x_size,h_size), dtype=tf.float64), dtype=tf.float64)
W2 = tf.Variable(initial_value=initial((h_size,y_size), dtype=tf.float64), dtype=tf.float64)

In [90]:
trainable = [W1, W2]

In [91]:
optimizer = tf.keras.optimizers.SGD(0.0001)

In [102]:
num_epochs = 101
rvalue = 0.1
for epoch in range(num_epochs):

    with tf.GradientTape() as tape:
        #yhat = forwardprop(X_normal,W1,W2)
        #predict = tf.argmax(yhat, axis=1)
        
        cost = ocnn_obj(theta, X_normal, nu, W1, W2, g, rvalue)
        grad_value = tape.gradient(cost, trainable)
    #rvalue = rvalue.eval()
    optimizer.apply_gradients(zip(grad_value, trainable))
    rvalue = nnScore(X_normal, W1, W2, g)
    
    
    rvalue = np.percentile(rvalue, q=100*nu)



    if epoch % 50 == 0:
        print("Epoch = %d, r = %f" % (epoch+1, rvalue))

Epoch = 1, r = 0.027193
Epoch = 51, r = 0.032044
Epoch = 101, r = 0.036870


In [103]:
train = nnScore(X_normal, W1, W2, g)
test = nnScore(X_anomaly, W1, W2, g)

rstar = rvalue

pos_decisionScore = train - rstar
neg_decisionScore = test - rstar

print("pos decisionScore\n", pos_decisionScore)
print("neg decisionScore\n", neg_decisionScore)

pos decisionScore
 tf.Tensor(
[[2.8603449e-03]
 [3.4704804e-05]
 [7.1860813e-03]
 ...
 [1.2916140e-03]
 [1.7439947e-04]
 [4.1470863e-03]], shape=(284315, 1), dtype=float32)
neg decisionScore
 tf.Tensor(
[[-1.50091946e-05]
 [ 9.92739573e-03]
 [ 4.58461046e-03]
 [ 1.11940876e-03]
 [ 9.89809632e-06]
 [ 9.55536962e-06]
 [ 9.67830420e-06]
 [ 9.55536962e-06]
 [ 9.76771116e-06]
 [ 3.79607081e-06]
 [ 4.03821468e-06]
 [ 4.04193997e-06]
 [ 4.33251262e-06]
 [ 4.30271029e-06]
 [ 9.92417336e-06]
 [ 9.66712832e-06]
 [ 9.56282020e-06]
 [ 4.10899520e-06]
 [ 3.68431211e-06]
 [ 4.36976552e-06]
 [ 4.21702862e-06]
 [ 4.09036875e-06]
 [ 1.00284815e-05]
 [ 9.97260213e-06]
 [ 2.71892063e-02]
 [ 8.94069672e-06]
 [ 9.73045826e-06]
 [ 3.05846334e-06]
 [ 9.26107168e-06]
 [ 9.88319516e-06]
 [-1.53817236e-05]
 [ 5.70561737e-04]
 [ 3.43832746e-03]
 [ 9.38400626e-06]
 [ 3.44589353e-06]
 [ 9.74908471e-06]
 [ 9.87574458e-06]
 [ 1.00731850e-05]
 [ 2.03389674e-04]
 [ 5.74029982e-05]
 [ 5.74551523e-05]
 [ 5.91427088e-05]

In [14]:
rvalue

340.1719970703125

In [125]:
threshold = -0.5

print("Normal Data : ", len(pos_decisionScore))
print("Normal - Detect Anomaly : ", np.sum(pos_decisionScore < threshold))
print("Anomaly Data : ", len(neg_decisionScore))
print("Anomaly - Detect Anomaly : ", np.sum(neg_decisionScore < threshold))

Normal Data :  284315
Normal - Detect Anomaly :  3902
Anomaly Data :  492
Anomaly - Detect Anomaly :  155


In [62]:
#minmax
#epoch300
#threshold = 1
threshold = 1


print("Normal Data : ", len(pos_decisionScore))
print("Normal - Detect Anomaly : ", np.sum(pos_decisionScore < threshold))
print("Anomaly Data : ", len(neg_decisionScore))
print("Anomaly - Detect Anomaly : ", np.sum(neg_decisionScore < threshold))

Normal Data :  284315
Normal - Detect Anomaly :  19553
Anomaly Data :  492
Anomaly - Detect Anomaly :  360


In [104]:
#noscaler
#epoch300
#threshold = 1
threshold = 0.0001


print("Normal Data : ", len(pos_decisionScore))
print("Normal - Detect Anomaly : ", np.sum(pos_decisionScore < threshold))
print("Anomaly Data : ", len(neg_decisionScore))
print("Anomaly - Detect Anomaly : ", np.sum(neg_decisionScore < threshold))

Normal Data :  284315
Normal - Detect Anomaly :  73607
Anomaly Data :  492
Anomaly - Detect Anomaly :  225


In [128]:
pos_decisionScore

<tf.Tensor: id=256571, shape=(284315, 1), dtype=float32, numpy=
array([[2.8582764],
       [0.7636719],
       [4.217407 ],
       ...,
       [2.0566711],
       [2.00885  ],
       [4.261688 ]], dtype=float32)>